In [1]:
# This cell is removed with the tag: "remove-input"
# As such, it will not be shown in documentation

# import warnings
# warnings.filterwarnings('ignore')

# Working with NGLView

MolSysMT can handle nglview.NGLWidget objects in the same way that it handles other molecular systems. Therefore, any MolSysMT tool can accept NGLView views as input systems.

## The nglview.NGLWidget as any other molecular system's form

Before we show some examples of how MolSysMT works with NGLView native objects, let's create an nglview.NGLWidget to play with. Let's work with the demo system from NGLView:

In [2]:
import molsysmt as msm
import nglview as nv

As first test, let's see if MolSysMT can get an information summary of the molecular system straight from the NGLView object:

In [ ]:
msm.info(view)

Attributes such as atom names, number of structures or atom coordinates can be retrieved with the function {func}`molsysmt.basic.get`:

In [ ]:
msm.get(view, element='group', selection=[81, 82, 83], name=True)

In [ ]:
msm.get(view, element='system', n_structures=True)

In [ ]:
msm.get(view, element='atom', selection='atom_name=="CA"', coordinates=True)

Element's selections can be made on NGLView native objects with MolSysMT:

In [ ]:
msm.select(view, selection='atom_name=="CA" and group_name=="LYS"')

In addition, MolSysMT can help you to get selection strings to be used in NGLView:

In [ ]:
msm.select(view, selection='atom_name=="CA" and group_name=="LYS"', to_syntax='nglview')

In [ ]:
msm.select(view, element='group', selection='group_name=="LYS"', to_syntax='nglview')

Can MolSysMT convert an nglview.NGLWidget to other molecular system forms? Have a look to this couple of examples:

In [ ]:
msm.convert(view, to_form='string:aminoacids3')

In [ ]:
msm.convert(view, to_form='openmm.Topology')

## Example 1: Getting contact maps

Now that it has been demonstrated that MolSysMT recognizes nglview.NGLWidget objects as any other molecular form. Let's show a couple of examples more appealing than the previous cells.

Let's compute and show a contact map between alpha carbons of the protein extracted from all frames in the NGLView native object:

In [ ]:
contact_map = msm.structure.get_contacts(view, selection='molecule_type=="protein" and atom_name=="CA"', threshold='12 angstroms')

In [ ]:
CA_labels = msm.get_label(view, selection='molecule_type=="protein" and atom_name=="CA"', string='{group_name}-{group_id}')

In [ ]:
# This cell is skipped with the cell tag "remove-cell"

#%matplotlib widget # This in the first cell of the notebook

import matplotlib.pyplot as plt
import matplotlib.animation as animation


plt.rcParams["animation.html"] = "jshtml"

fig = plt.figure()

structure_index = 0
im = plt.imshow(contact_map[structure_index], cmap='Greys', origin='lower', animated=True)
plt.xticks(range(0,contact_map.shape[1],25),CA_labels[::25], rotation=45)
plt.yticks(range(0,contact_map.shape[1],25),CA_labels[::25], rotation=45)

def updatefig(ii):
    global structure_index, contact_map
    structure_index = ii
    im.set_array(contact_map[structure_index])
    return im,

ani = animation.FuncAnimation(fig, updatefig, frames=contact_map.shape[0], blit=True, repeat=True)
#plt.show()
plt.close()
ani

In [ ]:
import plotly.express as px

fig = px.imshow(~contact_map, animation_frame=0, binary_string=True, height=600, origin='lower')

fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(0,contact_map.shape[1],25)),
        ticktext = CA_labels[::25],
        tickangle = 45
    ),
    yaxis = dict(
        tickmode = 'array',
        tickvals = list(range(0,contact_map.shape[1],25)),
        ticktext = CA_labels[::25],
        tickangle = 45
    )
)

fig.show()

In [ ]:
molsys_A = msm.build.build_peptide('AceAlaNME')
molsys_B = msm.structure.translate(molsys_A, translation='[0.5, 0.0, 0.0] nm')

In [ ]:
view1 = msm.convert(molsys_A, to_form='nglview.NGLWidget')
view2 = msm.view(molsys_B)

In [ ]:
view = msm.merge([view1, view2])

In [ ]:
msm.info(view)

In [ ]:
msm.structure.get_geometric_center(view, selection='molecule_index==0')

In [ ]:
msm.structure.get_geometric_center(view, selection='molecule_index==1')

In [ ]:
msm.structure.get_distances(view, selection='molecule_index==0', selection_2='molecule_index==1',
                            group_behavior='geometric center', group_behavior_2='geometric center')

In [ ]:
molecular_system = msm.convert([nv.datafiles.GRO, nv.datafiles.XTC], to_form='molsysmt.MolSys')

In [ ]:
view = nv.show_molsysmt(molecular_system)
view

A view was created with the ``nglview.show_molsysmt`` method, but it can also be produced straight from the ``molsysmt.basic.view`` native method:

In [ ]:
view = msm.view(molecular_system, standard=True)
view